In [ ]:
!pip install pyspark kafka-python pymongo matplotlib
!apt-get update
!apt-get install -y default-jdk

In [ ]:
from kafka import KafkaProducer, KafkaConsumer
from multiprocessing import Process
import json
import time
import random

def produce_data():
    producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                             value_serializer=lambda x: json.dumps(x).encode('utf-8'))

    while True:
        data = {
            'timestamp': int(time.time()),
            'sensor_id': f'sensor_{random.randint(1,5)}',
            'temperature': round(random.uniform(20, 30), 2),
            'humidity': round(random.uniform(30, 70), 2)
        }
        producer.send('sensor_data', value=data)
        time.sleep(1)

def consume_data():
    consumer = KafkaConsumer('sensor_data',
                             bootstrap_servers=['localhost:9092'],
                             value_deserializer=lambda x: json.loads(x.decode('utf-8')))

    for message in consumer:
        print(message.value)

# Start producer and consumer in separate processes
Process(target=produce_data).start()
Process(target=consume_data).start()

In [ ]:
# Initialize an empty DataFrame with the schema
df = spark.createDataFrame([], schema)

# Function to process each batch
def process_batch(batch_df):
    # Apply transformations
    processed_df = batch_df.filter(batch_df.value > 50).withColumn("timestamp", current_timestamp())
    processed_df.show(truncate=False)
    return processed_df

# Simulate streaming by processing data in a loop
for _ in range(5):  # Simulate 5 iterations/batches of data
    # Generate new batch of data
    batch_data = list(generate_data())
    batch_df = spark.createDataFrame(batch_data, schema=schema)

    # Process the batch
    processed_df = process_batch(batch_df)

    # Simulate time delay
    time.sleep(2)


In [ ]:
from pymongo import MongoClient

# Replace with your MongoDB connection string
mongo_uri = "your_mongo_uri"  # Replace with your MongoDB connection string

# Connect to MongoDB
client = MongoClient(mongo_uri)
db = client['streaming_db']
collection = db['processed_data']

# Function to write each batch to MongoDB
def write_to_mongo(batch_df):
    data = batch_df.toPandas().to_dict('records')
    if data:
        collection.insert_many(data)

# Storing the processed data into MongoDB
write_to_mongo(processed_df)


AnalysisException: [WRITE_STREAM_NOT_ALLOWED] `writeStream` can be called only on streaming Dataset/DataFrame.

In [ ]:
import matplotlib.pyplot as plt

# Sample data visualization
plt.figure(figsize=(10, 6))
plt.plot(processedDF.select("timestamp").collect(), processedDF.select("value").collect())
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Real-Time Data Visualization')
plt.show()


In [ ]:
def validate_data(batch_df):
    valid_df = batch_df.filter(batch_df.value.isNotNull())
    return valid_df

validatedDF = filteredDF.transform(validate_data)
